# Which

分类任务，支持2种模式
1. COCO数据集格式，具体格式说明：https://zhuanlan.zhihu.com/p/29393415
3. Image-Mask数据集格式，image一个文件夹，mask一个文件夹。同时一个seg_labels.txt文件，指定标签。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| FCN      | fcn_resnet50, fcn_resnet101                                                      |
| Deeplab          | deeplabv3_resnet50, deeplabv3_resnet101, deeplabv3_mobilenet_v3_large |
| lraspp       | lraspp_mobilenet_v3_large |
| Unet     | Unet           |

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

from onekey_algo.scripts.core import seg_covert2voc

train_dir = r'D:/20240320-RenJiLiang/json/train/'
val_dir = r'D:/20240320-RenJiLiang/json/test/'
save_dir = r'D:/20240320-RenJiLiang/endoscope_coco'
partition = [0.8, 0.2]
viz = True

seg_covert2voc(train_dir=train_dir, val_dir=[val_dir], save_dir = save_dir, noviz= not viz, del_directly=True)

# COCO

In [ ]:
from onekey_algo.segmentation.run_segmentation import main as seg_main
from onekey_algo import get_param_in_cwd
import os
# 如果自己有coco格式的数据，可以直接使用自己的目录。
my_dir = r'D:/20240320-RenJiLiang/endoscope_coco'

# 设置参数
for model_name in ['deeplabv3_resnet101', 'fcn_resnet101',  'NestedUNet', 'UNet', ]:
    class params:
        dataset = r'endoscope_coco_fmt'
        data_path = my_dir
        model = model_name
        lr = 0.01
        workers = get_param_in_cwd('workers', 4)
        batch_size = get_param_in_cwd('batch_size', 4)
        val_batch_size = 1
        print_freq = 8
        epochs = get_param_in_cwd('epochs', 30)
        optimizer = 'sgd'
        momentum = 0.9
        weight_decay = 1e-4
        save_dir = os.path.join(get_param_in_cwd('radio_dir'), 'Segmentation', 'endoscope_models')
        resume = r''
        dist_url = 'env://'
        world_size = 1e-4
        pretrained = True
        aux_loss = False
        test_only = False
        save_per_epoch = False
        downsample_ratio = get_param_in_cwd('downsample_ratio', 1)
        base_size = 512
        crop_size = 480
        attr = {}

        def __setattr__(self, key, value):
            self.attr[key] = value

    # 训练模型
    seg_main(params)

In [ ]:
import glob
import os
from onekey_algo.segmentation.eval_segmentation import init, inference

root = r'D:\20240320-RenJiLiang\Segmentation\endoscope_coco\models\Unet'
save_dir = os.path.join(root, 'predict')
model_root = os.path.join(root, 'viz')

test_samples = glob.glob(os.path.join(root, r'D:\20240320-RenJiLiang\Segmentation\endoscope_coco/*/JPEGImages/', '*.jpg'))

model, class_names, device = init(model_root)
inference(test_samples, model, device=device, class_names=class_names, save_dir=save_dir)

In [ ]:
import glob
import numpy as np

def calc_dice(p_cls, l_cls):
    # cal the inter & conv
    s = p_cls + l_cls
    inter = len(np.where(s >= 2)[0])
    conv = len(np.where(s >= 1)[0]) + inter
    try:
        dice = 2.0 * inter / conv
    except:
        print("conv is zeros when dice = 2.0 * inter / conv")
        dice = None
    return dice

def calc_iou(p_cls, l_cls):
    # cal the inter & conv
    s = p_cls + l_cls
    inter = len(np.where(s >= 2)[0])
    conv = len(np.where(s >= 1)[0])
    try:
        iou = inter / conv
    except:
        print("conv is zeros when dice = 2.0 * inter / conv")
        iou = None
    return iou

In [ ]:
import pandas as pd
group_info = pd.read_csv('group.csv')
group_info['ID'] = group_info['ID'].map(lambda x: x.replace('.nii.gz', '.png'))
group_info

In [ ]:
from PIL import Image
import pandas as pd
import os

details = []
for model_name in ['deeplabv3_resnet101', 'fcn_resnet101', 'UNet', ]:
    for subset in ['train', 'val', 'test']:
        samples = glob.glob(os.path.join(rf'Y:\20240320-RenJiLiang\Segmentation\endoscope_coco/*/Masks/', '*.png'))
        sset = set(group_info[group_info['group'] == subset]['ID'])
        for sample in [s for s in samples if os.path.basename(s) in sset]:
            simg = Image.open(sample)
            infer = os.path.join(r'Y:\20240320-RenJiLiang\Segmentation\endoscope_coco\models', model_name, 'predict', 'masks',
                                 os.path.basename(sample))
            iimg = Image.open(infer)
            dice = calc_dice(np.array(iimg), np.array(simg))
            iou = calc_iou(np.array(iimg), np.array(simg))
            details.append([os.path.basename(sample), dice, iou, model_name, subset])
details = pd.DataFrame(details, columns=['sample', 'Dice', 'mIoU', 'Model', 'Cohort'])
details.to_csv('data/endoscope.csv', index=False)
details

In [ ]:
details.groupby(['Model', 'Cohort']).aggregate('mean').reset_index()